# Getting started with TinyTimeMixer (TTM)

This notebooke demonstrates the usage of a pre-trained `TinyTimeMixer` model for several multivariate time series forecasting tasks. For details related to model architecture, refer to the [TTM paper](https://arxiv.org/pdf/2401.03955.pdf).

In this example, we will use a pre-trained TTM-512-96 model. That means the TTM model can take an input of 512 time points (`context_length`), and can forecast upto 96 time points (`forecast_length`) in the future. We will use the pre-trained TTM in two settings:
1. **Zero-shot**: The pre-trained TTM will be directly used to evaluate on the `test` split of the target data. Note that the TTM was NOT pre-trained on the target data.
2. **Few-shot**: The pre-trained TTM will be quickly fine-tuned on only 5% of the `train` split of the target data, and subsequently, evaluated on the `test` part of the target data.

Note: Alternatively, this notebook can be modified to try the TTM-1024-96 model.

Pre-trained TTM models will be fetched from the [Hugging Face TTM Model Repository](https://huggingface.co/ibm/TTM).

## Installation

In [1]:
# Clone the ibm/tsfm
!git clone https://github.com/IBM/tsfm.git

Cloning into 'tsfm'...
remote: Enumerating objects: 14616, done.
remote: Counting objects: 100% (3062/3062), done.
remote: Compressing objects: 100% (444/444), done.
remote: Total 14616 (delta 2814), reused 2666 (delta 2608), pack-reused 11554 (from 2)
Receiving objects: 100% (14616/14616), 93.73 MiB | 20.97 MiB/s, done.
Resolving deltas: 100% (10011/10011), done.


In [2]:
# Change directory. Move inside the tsfm repo.
%cd tsfm

/content/tsfm


In [3]:
# Do ls
! ls

agentic  Makefile   pyproject.toml  README.md  services  tsfmhfdemos  uv.lock
LICENSE  notebooks  pytest.ini	    scripts    tests	 tsfm_public  wiki.md


In [4]:
# Install the tsfm library
! pip install ".[notebooks]"

Processing /content/tsfm
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 90.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.9/887.9 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 49.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 MB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 954.8/954.8 kB 24.5 MB/s eta 0:00:00
   ━━━━━━╺━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.4/193.1 MB 111.0 MB/s eta 0:00:02
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 21.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.6/63.6 MB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26

In [5]:
!pip uninstall -y transformers peft accelerate sentence-transformers
!pip install "transformers==4.41.0" "accelerate<1.0.0"
!pip install "granite-tsfm[notebooks] @ git+https://github.com/ibm-granite/granite-tsfm.git@v0.2.22"


Found existing installation: transformers 4.57.6
Uninstalling transformers-4.57.6:
  Successfully uninstalled transformers-4.57.6
Found existing installation: peft 0.18.1
Uninstalling peft-0.18.1:
  Successfully uninstalled peft-0.18.1
Found existing installation: accelerate 1.12.0
Uninstalling accelerate-1.12.0:
  Successfully uninstalled accelerate-1.12.0
Found existing installation: sentence-transformers 5.2.2
Uninstalling sentence-transformers-5.2.2:
  Successfully uninstalled sentence-transformers-5.2.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 40.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 324.4/324.4 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 54.1 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.22.2
    Uninstalling tokenizers-0.22.2:
      Successfully uninstalled tokenizers-0.22.2


In [1]:
import transformers, accelerate
print("transformers:", transformers.__version__)
print("accelerate:", accelerate.__version__)


transformers: 4.41.0
accelerate: 0.34.2


In [2]:
from tsfm_public.toolkit.get_model import get_model
from tsfm_public import TimeSeriesPreprocessor, get_datasets
print("TSFM imports OK ✅")


TSFM imports OK ✅


In [3]:
TTM_MODEL_PATH = "ibm-granite/granite-timeseries-ttm-r2"
model = get_model(TTM_MODEL_PATH, context_length=60, prediction_length=7)
print("Model loaded ✅", model.config.context_length, model.config.prediction_length)


INFO:/usr/local/lib/python3.12/dist-packages/tsfm_public/toolkit/get_model.py:Loading model from: ibm-granite/granite-timeseries-ttm-r2
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/319k [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/69.0 [00:00<?, ?B/s]

INFO:/usr/local/lib/python3.12/dist-packages/tsfm_public/toolkit/get_model.py:Model loaded successfully from ibm-granite/granite-timeseries-ttm-r2, revision = 52-16-ft-r2.1.
INFO:/usr/local/lib/python3.12/dist-packages/tsfm_public/toolkit/get_model.py:[TTM] context_length = 52, prediction_length = 16


Model loaded ✅ 52 16


In [4]:
import pandas as pd

# Loading Dataset
csv_path = "2NewCombined.csv"

df = pd.read_csv(csv_path)

print("Rows:", len(df))
print("Columns:")
for c in df.columns:
    print("-", c)

df.head()


Rows: 183
Columns:
- Date
- Daily Rainfall Total (mm)_Admiralty
- Daily Rainfall Total (mm)_Ang Mo Kio
- Daily Rainfall Total (mm)_Changi
- Daily Rainfall Total (mm)_Sentosa Island
- Daily Rainfall Total (mm)_Tuas South
- Highest 120 min Rainfall (mm)_Admiralty
- Highest 120 min Rainfall (mm)_Ang Mo Kio
- Highest 120 min Rainfall (mm)_Changi
- Highest 120 min Rainfall (mm)_Sentosa Island
- Highest 120 min Rainfall (mm)_Tuas South
- Highest 30 min Rainfall (mm)_Admiralty
- Highest 30 min Rainfall (mm)_Ang Mo Kio
- Highest 30 min Rainfall (mm)_Changi
- Highest 30 min Rainfall (mm)_Sentosa Island
- Highest 30 min Rainfall (mm)_Tuas South
- Highest 60 min Rainfall (mm)_Admiralty
- Highest 60 min Rainfall (mm)_Ang Mo Kio
- Highest 60 min Rainfall (mm)_Changi
- Highest 60 min Rainfall (mm)_Sentosa Island
- Highest 60 min Rainfall (mm)_Tuas South
- Max Wind Speed (km/h)_Admiralty
- Max Wind Speed (km/h)_Ang Mo Kio
- Max Wind Speed (km/h)_Changi
- Max Wind Speed (km/h)_Sentosa Island
- Max Win

,Date,Daily Rainfall Total (mm)_Admiralty,Daily Rainfall Total (mm)_Ang Mo Kio,Daily Rainfall Total (mm)_Changi,Daily Rainfall Total (mm)_Sentosa Island,Daily Rainfall Total (mm)_Tuas South,Highest 120 min Rainfall (mm)_Admiralty,Highest 120 min Rainfall (mm)_Ang Mo Kio,Highest 120 min Rainfall (mm)_Changi,Highest 120 min Rainfall (mm)_Sentosa Island,...,Mean Wind Speed (km/h)_Admiralty,Mean Wind Speed (km/h)_Ang Mo Kio,Mean Wind Speed (km/h)_Changi,Mean Wind Speed (km/h)_Sentosa Island,Mean Wind Speed (km/h)_Tuas South,Minimum Temperature (°C)_Admiralty,Minimum Temperature (°C)_Ang Mo Kio,Minimum Temperature (°C)_Changi,Minimum Temperature (°C)_Sentosa Island,Minimum Temperature (°C)_Tuas South
0,1/4/2025,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,7.2,5.1,10.6,5.6,6.4,25.3,25.6,25.1,25.9,25.7
1,1/5/2025,25.8,40.4,11.0,0.0,6.4,25.8,39.8,5.0,0.0,...,6.1,3.6,5.4,4.5,5.6,24.4,24.8,24.9,26.3,26.1
2,1/6/2025,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,8.0,6.1,12.1,5.5,6.7,26.6,27.1,27.4,26.8,28.3
3,1/7/2025,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,9.3,6.0,10.3,5.6,5.5,25.9,27.6,27.2,27.6,28.5
4,1/8/2025,0.2,0.0,0.0,0.0,0.0,0.2,0.0,0.0,0.0,...,6.9,6.1,12.3,6.0,5.8,26.9,28.4,27.9,28.2,28.6


In [5]:
import os
import pandas as pd

print("Current folder:", os.getcwd())
print("Files here:", os.listdir())

df = pd.read_csv("2NewCombined.csv")
print("\n Loaded file")
print("Rows:", len(df))
print("First 15 columns:", df.columns.tolist()[:15])

df.head()


Current folder: /content
Files here: ['.config', 'Current Codes FYP.zip', '2NewCombined.csv', 'tsfm', 'sample_data']

 Loaded file
Rows: 183
First 15 columns: ['Date', 'Daily Rainfall Total (mm)_Admiralty', 'Daily Rainfall Total (mm)_Ang Mo Kio', 'Daily Rainfall Total (mm)_Changi', 'Daily Rainfall Total (mm)_Sentosa Island', 'Daily Rainfall Total (mm)_Tuas South', 'Highest 120 min Rainfall (mm)_Admiralty', 'Highest 120 min Rainfall (mm)_Ang Mo Kio', 'Highest 120 min Rainfall (mm)_Changi', 'Highest 120 min Rainfall (mm)_Sentosa Island', 'Highest 120 min Rainfall (mm)_Tuas South', 'Highest 30 min Rainfall (mm)_Admiralty', 'Highest 30 min Rainfall (mm)_Ang Mo Kio', 'Highest 30 min Rainfall (mm)_Changi', 'Highest 30 min Rainfall (mm)_Sentosa Island']


,Date,Daily Rainfall Total (mm)_Admiralty,Daily Rainfall Total (mm)_Ang Mo Kio,Daily Rainfall Total (mm)_Changi,Daily Rainfall Total (mm)_Sentosa Island,Daily Rainfall Total (mm)_Tuas South,Highest 120 min Rainfall (mm)_Admiralty,Highest 120 min Rainfall (mm)_Ang Mo Kio,Highest 120 min Rainfall (mm)_Changi,Highest 120 min Rainfall (mm)_Sentosa Island,...,Mean Wind Speed (km/h)_Admiralty,Mean Wind Speed (km/h)_Ang Mo Kio,Mean Wind Speed (km/h)_Changi,Mean Wind Speed (km/h)_Sentosa Island,Mean Wind Speed (km/h)_Tuas South,Minimum Temperature (°C)_Admiralty,Minimum Temperature (°C)_Ang Mo Kio,Minimum Temperature (°C)_Changi,Minimum Temperature (°C)_Sentosa Island,Minimum Temperature (°C)_Tuas South
0,1/4/2025,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,7.2,5.1,10.6,5.6,6.4,25.3,25.6,25.1,25.9,25.7
1,1/5/2025,25.8,40.4,11.0,0.0,6.4,25.8,39.8,5.0,0.0,...,6.1,3.6,5.4,4.5,5.6,24.4,24.8,24.9,26.3,26.1
2,1/6/2025,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,8.0,6.1,12.1,5.5,6.7,26.6,27.1,27.4,26.8,28.3
3,1/7/2025,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,9.3,6.0,10.3,5.6,5.5,25.9,27.6,27.2,27.6,28.5
4,1/8/2025,0.2,0.0,0.0,0.0,0.0,0.2,0.0,0.0,0.0,...,6.9,6.1,12.3,6.0,5.8,26.9,28.4,27.9,28.2,28.6


In [6]:
from tsfm_public import TimeSeriesPreprocessor

CONTEXT_LENGTH = 52
PREDICTION_LENGTH = 7

column_specifiers = {
    "timestamp_column": "Date",
    "id_columns": [],
    "target_columns": [
        "Daily Rainfall Total (mm)_Ang Mo Kio",
        "Mean Wind Speed (km/h)_Ang Mo Kio",
        "Minimum Temperature (°C)_Ang Mo Kio",
    ],
    "control_columns": [],
}

tsp = TimeSeriesPreprocessor(
    **column_specifiers,
    context_length=CONTEXT_LENGTH,
    prediction_length=PREDICTION_LENGTH,
    scaling=True,
    encode_categorical=False,
    scaler_type="standard",
)

print("Preprocessor created successfully")


Preprocessor created successfully


In [ ]:
import pandas as pd
import tempfile

from transformers import Trainer, TrainingArguments, set_seed
from tsfm_public.toolkit.get_model import get_model
from tsfm_public import TimeSeriesPreprocessor, get_datasets

set_seed(42)

# Load data (with fixed date parsing)
df = pd.read_csv("2NewCombined.csv")
df["Date"] = pd.to_datetime(df["Date"], dayfirst=True, format="mixed")
df = df.sort_values("Date").reset_index(drop=True)

# Keep only needed columns
df = df[
    [
        "Date",
        "Daily Rainfall Total (mm)_Ang Mo Kio",
        "Mean Wind Speed (km/h)_Ang Mo Kio",
        "Minimum Temperature (°C)_Ang Mo Kio",
    ]
]

# Split (70 / 15 / 15)
n = len(df)
train_end = int(n * 0.70)
valid_end = int(n * 0.85)

split_config = {
    "train": [0, train_end],
    "valid": [train_end, valid_end],
    "test":  [valid_end, n],
}

# TTM settings
TTM_MODEL_PATH = "ibm-granite/granite-timeseries-ttm-r2"
CONTEXT_LENGTH = 52
PREDICTION_LENGTH = 7

column_specifiers = {
    "timestamp_column": "Date",
    "id_columns": [],
    "target_columns": [
        "Daily Rainfall Total (mm)_Ang Mo Kio",
        "Mean Wind Speed (km/h)_Ang Mo Kio",
        "Minimum Temperature (°C)_Ang Mo Kio",
    ],
    "control_columns": [],
}

# Preprocessor
tsp = TimeSeriesPreprocessor(
    **column_specifiers,
    context_length=CONTEXT_LENGTH,
    prediction_length=PREDICTION_LENGTH,
    scaling=True,
    encode_categorical=False,
    scaler_type="standard",
)

# Zero-shot model
model = get_model(
    TTM_MODEL_PATH,
    context_length=CONTEXT_LENGTH,
    prediction_length=PREDICTION_LENGTH,
    freq_prefix_tuning=False,
    freq=None,
    prefer_longer_context=True,
)

# Datasets
dset_train, dset_valid, dset_test = get_datasets(
    tsp, df, split_config, use_frequency_token=model.config.resolution_prefix_tuning
)

# Trainer (evaluation only)
temp_dir = tempfile.mkdtemp()
trainer = Trainer(
    model=model,
    args=TrainingArguments(
        output_dir=temp_dir,
        per_device_eval_batch_size=32,
        report_to="none",
        seed=42,
    ),
)

print("+" * 20, "Zero-shot TEST (MSE)", "+" * 20)
metrics = trainer.evaluate(dset_test)
print(metrics)

preds = trainer.predict(dset_test).predictions[0]
print("✅ Predictions shape:", preds.shape)


In [ ]:
# ======================================================
# ZERO-SHOT TTM (AMK WEATHER)
# Metrics: MSE, R², Pearson p-value
# Includes graphs (Granite-style)
# ======================================================

import os
import numpy as np
import pandas as pd
import tempfile

from sklearn.metrics import mean_squared_error, r2_score
from scipy.stats import pearsonr

from transformers import Trainer, TrainingArguments, set_seed
from tsfm_public.toolkit.get_model import get_model
from tsfm_public import TimeSeriesPreprocessor, get_datasets
from tsfm_public.toolkit.visualization import plot_predictions

# ---------------------------
# REPRODUCIBILITY
# ---------------------------
set_seed(42)

# ---------------------------
# LOAD + CLEAN DATA
# ---------------------------
df = pd.read_csv("2NewCombined.csv")

# Fix Singapore-style dates (dd/mm/yyyy)
df["Date"] = pd.to_datetime(df["Date"], dayfirst=True, format="mixed")
df = df.sort_values("Date").reset_index(drop=True)

# Use AMK only (clear + defensible)
df = df[
    [
        "Date",
        "Daily Rainfall Total (mm)_Ang Mo Kio",
        "Mean Wind Speed (km/h)_Ang Mo Kio",
        "Minimum Temperature (°C)_Ang Mo Kio",
    ]
]

# ---------------------------
# TRAIN / VALID / TEST SPLIT
# 70% / 15% / 15%
# ---------------------------
n = len(df)
train_end = int(n * 0.70)
valid_end = int(n * 0.85)

split_config = {
    "train": [0, train_end],
    "valid": [train_end, valid_end],
    "test":  [valid_end, n],
}

print("Split sizes:")
print("Train:", train_end)
print("Valid:", valid_end - train_end)
print("Test :", n - valid_end)

# ---------------------------
# TTM SETTINGS (ZERO-SHOT)
# ---------------------------
TTM_MODEL_PATH = "ibm-granite/granite-timeseries-ttm-r2"
CONTEXT_LENGTH = 52
PREDICTION_LENGTH = 7

column_specifiers = {
    "timestamp_column": "Date",
    "id_columns": [],
    "target_columns": [
        "Daily Rainfall Total (mm)_Ang Mo Kio",
        "Mean Wind Speed (km/h)_Ang Mo Kio",
        "Minimum Temperature (°C)_Ang Mo Kio",
    ],
    "control_columns": [],
}

# ---------------------------
# PREPROCESSING
# ---------------------------
tsp = TimeSeriesPreprocessor(
    **column_specifiers,
    context_length=CONTEXT_LENGTH,
    prediction_length=PREDICTION_LENGTH,
    scaling=True,
    encode_categorical=False,
    scaler_type="standard",
)

# ---------------------------
# LOAD PRETRAINED MODEL
# ---------------------------
model = get_model(
    TTM_MODEL_PATH,
    context_length=CONTEXT_LENGTH,
    prediction_length=PREDICTION_LENGTH,
    prefer_longer_context=True,
)

# ---------------------------
# BUILD DATASETS
# ---------------------------
dset_train, dset_valid, dset_test = get_datasets(
    tsp,
    df,
    split_config,
    use_frequency_token=model.config.resolution_prefix_tuning,
)

# ---------------------------
# ZERO-SHOT EVALUATION
# ---------------------------
trainer = Trainer(
    model=model,
    args=TrainingArguments(
        output_dir=tempfile.mkdtemp(),
        per_device_eval_batch_size=32,
        report_to="none",
        seed=42,
    ),
)

print("\nZERO-SHOT TEST (MSE from Trainer)")
zero_metrics = trainer.evaluate(dset_test)
print(zero_metrics)

# ---------------------------
# PREDICTIONS
# ---------------------------
preds = trainer.predict(dset_test).predictions[0]  # (samples, 7, 3)
print("Predictions shape:", preds.shape)

# ---------------------------
# GET GROUND TRUTH SAFELY
# ---------------------------
# TTM datasets store future values under different keys depending on config
sample = dset_test[0]
print("Dataset keys:", sample.keys())

if "labels" in sample:
    true = np.array([x["labels"] for x in dset_test])
elif "future_values" in sample:
    true = np.array([x["future_values"] for x in dset_test])
elif "y" in sample:
    true = np.array([x["y"] for x in dset_test])
else:
    raise KeyError("Cannot find ground-truth values in dataset")

# ---------------------------
# METRICS (RAIN ONLY – CHANNEL 0)
# ---------------------------
y_true = true[:, :, 0].flatten()
y_pred = preds[:, :, 0].flatten()

mse = mean_squared_error(y_true, y_pred)
r2 = r2_score(y_true, y_pred)
corr, p_value = pearsonr(y_true, y_pred)

print("\nZERO-SHOT METRICS (Rainfall – Ang Mo Kio)")
print(f"MSE     : {mse:.4f}")
print(f"R²      : {r2:.4f}")
print(f"p-value : {p_value:.6f}")

# ---------------------------
# PLOTS (LIKE GRANITE NOTEBOOK)
# ---------------------------
PLOT_DIR = "ttm_zero_shot_plots"
os.makedirs(PLOT_DIR, exist_ok=True)

# Ensure indices are valid
max_idx = len(dset_test) - 1
plot_indices = [i for i in [0, 5, 10] if i <= max_idx]
if not plot_indices:
    plot_indices = [0]

plot_predictions(
    model=trainer.model,
    dset=dset_test,
    plot_dir=PLOT_DIR,
    plot_prefix="amk_zero_shot_rainfall",
    indices=plot_indices,
    channel=0,  # rainfall
)

print("\n✅ Zero-shot rainfall plots saved to:", PLOT_DIR)
print("Open the left Files panel →", PLOT_DIR)


Few Shots 5%

In [ ]:
# ======================================================
# FEW-SHOT TTM (5%) — AMK WEATHER
# Metrics: MSE, R², Pearson p-value
# Includes graphs (same style as zero-shot)
# ======================================================

import os
import math
import numpy as np
import pandas as pd
import tempfile

from sklearn.metrics import mean_squared_error, r2_score
from scipy.stats import pearsonr

from torch.optim import AdamW
from torch.optim.lr_scheduler import OneCycleLR
from transformers import (
    Trainer,
    TrainingArguments,
    EarlyStoppingCallback,
    set_seed,
)
from transformers.integrations import INTEGRATION_TO_CALLBACK

from tsfm_public.toolkit.get_model import get_model
from tsfm_public import TimeSeriesPreprocessor, get_datasets
from tsfm_public.toolkit.visualization import plot_predictions
from tsfm_public import TrackingCallback, count_parameters

# ---------------------------
# REPRODUCIBILITY
# ---------------------------
set_seed(42)

# ---------------------------
# LOAD + CLEAN DATA
# ---------------------------
df = pd.read_csv("2NewCombined.csv")
df["Date"] = pd.to_datetime(df["Date"], dayfirst=True, format="mixed")
df = df.sort_values("Date").reset_index(drop=True)

df = df[
    [
        "Date",
        "Daily Rainfall Total (mm)_Ang Mo Kio",
        "Mean Wind Speed (km/h)_Ang Mo Kio",
        "Minimum Temperature (°C)_Ang Mo Kio",
    ]
]

# ---------------------------
# TRAIN / VALID / TEST SPLIT
# ---------------------------
n = len(df)
train_end = int(n * 0.70)
valid_end = int(n * 0.85)

split_config = {
    "train": [0, train_end],
    "valid": [train_end, valid_end],
    "test":  [valid_end, n],
}

# ---------------------------
# TTM SETTINGS (FEW-SHOT)
# ---------------------------
TTM_MODEL_PATH = "ibm-granite/granite-timeseries-ttm-r2"
CONTEXT_LENGTH = 52
PREDICTION_LENGTH = 7
FEWSHOT_PERCENT = 5

column_specifiers = {
    "timestamp_column": "Date",
    "id_columns": [],
    "target_columns": [
        "Daily Rainfall Total (mm)_Ang Mo Kio",
        "Mean Wind Speed (km/h)_Ang Mo Kio",
        "Minimum Temperature (°C)_Ang Mo Kio",
    ],
    "control_columns": [],
}

# ---------------------------
# PREPROCESSING
# ---------------------------
tsp = TimeSeriesPreprocessor(
    **column_specifiers,
    context_length=CONTEXT_LENGTH,
    prediction_length=PREDICTION_LENGTH,
    scaling=True,
    encode_categorical=False,
    scaler_type="standard",
)

# ---------------------------
# LOAD MODEL (FOR FINE-TUNING)
# ---------------------------
model = get_model(
    TTM_MODEL_PATH,
    context_length=CONTEXT_LENGTH,
    prediction_length=PREDICTION_LENGTH,
    prefer_longer_context=True,
    loss="mse",
)

# ---------------------------
# FEW-SHOT DATASETS (5% OF TRAIN)
# ---------------------------
dset_train_fs, dset_valid_fs, dset_test_fs = get_datasets(
    tsp,
    df,
    split_config,
    fewshot_fraction=FEWSHOT_PERCENT / 100,
    fewshot_location="first",
    use_frequency_token=model.config.resolution_prefix_tuning,
)

# ---------------------------
# FREEZE BACKBONE (IMPORTANT)
# ---------------------------
print("Params before freezing:", count_parameters(model))
for p in model.backbone.parameters():
    p.requires_grad = False
print("Params after freezing:", count_parameters(model))

# ---------------------------
# TRAINING SETUP
# ---------------------------
args = TrainingArguments(
    output_dir="ttm_fewshot_output",
    overwrite_output_dir=True,
    learning_rate=1e-3,
    num_train_epochs=50,
    do_eval=True,
    eval_strategy="epoch",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    report_to="none",
    save_strategy="epoch",
    save_total_limit=1,
    logging_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    seed=42,
)

early_stop = EarlyStoppingCallback(
    early_stopping_patience=10,
    early_stopping_threshold=1e-5,
)
tracking = TrackingCallback()

optimizer = AdamW(model.parameters(), lr=1e-3)
scheduler = OneCycleLR(
    optimizer,
    max_lr=1e-3,
    epochs=args.num_train_epochs,
    steps_per_epoch=math.ceil(len(dset_train_fs) / args.per_device_train_batch_size),
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=dset_train_fs,
    eval_dataset=dset_valid_fs,
    callbacks=[early_stop, tracking],
    optimizers=(optimizer, scheduler),
)

# Remove optional integration callback if present
trainer.remove_callback(INTEGRATION_TO_CALLBACK["codecarbon"])

# ---------------------------
# TRAIN
# ---------------------------
print("\n--- FEW-SHOT TRAINING (5%) ---")
trainer.train()

# ---------------------------
# EVALUATION
# ---------------------------
print("\nFEW-SHOT TEST (MSE from Trainer)")
fewshot_metrics = trainer.evaluate(dset_test_fs)
print(fewshot_metrics)

# ---------------------------
# PREDICTIONS
# ---------------------------
preds = trainer.predict(dset_test_fs).predictions[0]
print("Predictions shape:", preds.shape)

# ---------------------------
# GET GROUND TRUTH SAFELY
# ---------------------------
sample = dset_test_fs[0]
if "labels" in sample:
    true = np.array([x["labels"] for x in dset_test_fs])
elif "future_values" in sample:
    true = np.array([x["future_values"] for x in dset_test_fs])
elif "y" in sample:
    true = np.array([x["y"] for x in dset_test_fs])
else:
    raise KeyError("Cannot find ground-truth values")

# ---------------------------
# METRICS (RAIN ONLY)
# ---------------------------
y_true = true[:, :, 0].flatten()
y_pred = preds[:, :, 0].flatten()

mse = mean_squared_error(y_true, y_pred)
r2 = r2_score(y_true, y_pred)
corr, p_value = pearsonr(y_true, y_pred)

print("\nFEW-SHOT METRICS (Rainfall – Ang Mo Kio)")
print(f"MSE     : {mse:.4f}")
print(f"R²      : {r2:.4f}")
print(f"p-value : {p_value:.6f}")

# ---------------------------
# PLOTS
# ---------------------------
PLOT_DIR = "ttm_fewshot_plots"
os.makedirs(PLOT_DIR, exist_ok=True)

max_idx = len(dset_test_fs) - 1
plot_indices = [i for i in [0, 5, 10] if i <= max_idx]
if not plot_indices:
    plot_indices = [0]

plot_predictions(
    model=trainer.model,
    dset=dset_test_fs,
    plot_dir=PLOT_DIR,
    plot_prefix="amk_fewshot_rainfall",
    indices=plot_indices,
    channel=0,
)

print("\n✅ Few-shot rainfall plots saved to:", PLOT_DIR)


Few Shots and Quantile Loss

In [ ]:
# ======================================================
# FEW-SHOT TTM (5%) — QUANTILE / PINBALL LOSS
# Quantile: q = 0.5 (median)
# Includes metrics + graphs
# ======================================================

import os
import math
import numpy as np
import pandas as pd
import tempfile

from sklearn.metrics import mean_squared_error, r2_score
from scipy.stats import pearsonr

from torch.optim import AdamW
from torch.optim.lr_scheduler import OneCycleLR
from transformers import (
    Trainer,
    TrainingArguments,
    EarlyStoppingCallback,
    set_seed,
)
from transformers.integrations import INTEGRATION_TO_CALLBACK

from tsfm_public.toolkit.get_model import get_model
from tsfm_public import TimeSeriesPreprocessor, get_datasets
from tsfm_public.toolkit.visualization import plot_predictions
from tsfm_public import TrackingCallback, count_parameters

# ---------------------------
# REPRODUCIBILITY
# ---------------------------
set_seed(42)

# ---------------------------
# LOAD + CLEAN DATA
# ---------------------------
df = pd.read_csv("2NewCombined.csv")
df["Date"] = pd.to_datetime(df["Date"], dayfirst=True, format="mixed")
df = df.sort_values("Date").reset_index(drop=True)

df = df[
    [
        "Date",
        "Daily Rainfall Total (mm)_Ang Mo Kio",
        "Mean Wind Speed (km/h)_Ang Mo Kio",
        "Minimum Temperature (°C)_Ang Mo Kio",
    ]
]

# ---------------------------
# TRAIN / VALID / TEST SPLIT
# ---------------------------
n = len(df)
train_end = int(n * 0.70)
valid_end = int(n * 0.85)

split_config = {
    "train": [0, train_end],
    "valid": [train_end, valid_end],
    "test":  [valid_end, n],
}

# ---------------------------
# TTM SETTINGS
# ---------------------------
TTM_MODEL_PATH = "ibm-granite/granite-timeseries-ttm-r2"
CONTEXT_LENGTH = 52
PREDICTION_LENGTH = 7
FEWSHOT_PERCENT = 5
QUANTILE = 0.5   # median forecast

column_specifiers = {
    "timestamp_column": "Date",
    "id_columns": [],
    "target_columns": [
        "Daily Rainfall Total (mm)_Ang Mo Kio",
        "Mean Wind Speed (km/h)_Ang Mo Kio",
        "Minimum Temperature (°C)_Ang Mo Kio",
    ],
    "control_columns": [],
}

# ---------------------------
# PREPROCESSING
# ---------------------------
tsp = TimeSeriesPreprocessor(
    **column_specifiers,
    context_length=CONTEXT_LENGTH,
    prediction_length=PREDICTION_LENGTH,
    scaling=True,
    encode_categorical=False,
    scaler_type="standard",
)

# ---------------------------
# LOAD MODEL (QUANTILE LOSS)
# ---------------------------
model = get_model(
    TTM_MODEL_PATH,
    context_length=CONTEXT_LENGTH,
    prediction_length=PREDICTION_LENGTH,
    prefer_longer_context=True,
    loss="pinball",
    quantile=QUANTILE,
)

# ---------------------------
# FEW-SHOT DATASETS (5%)
# ---------------------------
dset_train_fs, dset_valid_fs, dset_test_fs = get_datasets(
    tsp,
    df,
    split_config,
    fewshot_fraction=FEWSHOT_PERCENT / 100,
    fewshot_location="first",
    use_frequency_token=model.config.resolution_prefix_tuning,
)

# ---------------------------
# FREEZE BACKBONE
# ---------------------------
print("Params before freezing:", count_parameters(model))
for p in model.backbone.parameters():
    p.requires_grad = False
print("Params after freezing:", count_parameters(model))

# ---------------------------
# TRAINING SETUP
# ---------------------------
args = TrainingArguments(
    output_dir="ttm_fewshot_quantile_output",
    overwrite_output_dir=True,
    learning_rate=1e-3,
    num_train_epochs=50,
    do_eval=True,
    eval_strategy="epoch",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    report_to="none",
    save_strategy="epoch",
    save_total_limit=1,
    logging_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",  # pinball loss
    greater_is_better=False,
    seed=42,
)

early_stop = EarlyStoppingCallback(
    early_stopping_patience=10,
    early_stopping_threshold=1e-5,
)
tracking = TrackingCallback()

optimizer = AdamW(model.parameters(), lr=1e-3)
scheduler = OneCycleLR(
    optimizer,
    max_lr=1e-3,
    epochs=args.num_train_epochs,
    steps_per_epoch=math.ceil(len(dset_train_fs) / args.per_device_train_batch_size),
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=dset_train_fs,
    eval_dataset=dset_valid_fs,
    callbacks=[early_stop, tracking],
    optimizers=(optimizer, scheduler),
)

trainer.remove_callback(INTEGRATION_TO_CALLBACK["codecarbon"])

# ---------------------------
# TRAIN
# ---------------------------
print("\n--- FEW-SHOT TRAINING (5%, QUANTILE LOSS) ---")
trainer.train()

# ---------------------------
# EVALUATION (PINBALL LOSS)
# ---------------------------
print("\nFEW-SHOT TEST (Pinball loss)")
fewshot_metrics = trainer.evaluate(dset_test_fs)
print(fewshot_metrics)

# ---------------------------
# PREDICTIONS
# ---------------------------
preds = trainer.predict(dset_test_fs).predictions[0]
print("Predictions shape:", preds.shape)

# ---------------------------
# GET GROUND TRUTH SAFELY
# ---------------------------
sample = dset_test_fs[0]
if "labels" in sample:
    true = np.array([x["labels"] for x in dset_test_fs])
elif "future_values" in sample:
    true = np.array([x["future_values"] for x in dset_test_fs])
elif "y" in sample:
    true = np.array([x["y"] for x in dset_test_fs])
else:
    raise KeyError("Cannot find ground-truth values")

# ---------------------------
# OPTIONAL: MSE & R² (RAIN ONLY)
# (for comparison only — NOT primary metric)
# ---------------------------
y_true = true[:, :, 0].flatten()
y_pred = preds[:, :, 0].flatten()

mse = mean_squared_error(y_true, y_pred)
r2 = r2_score(y_true, y_pred)
corr, p_value = pearsonr(y_true, y_pred)

print("\nFEW-SHOT QUANTILE METRICS (Rainfall – comparison only)")
print(f"MSE     : {mse:.4f}")
print(f"R²      : {r2:.4f}")
print(f"p-value : {p_value:.6f}")

# ---------------------------
# PLOTS
# ---------------------------
PLOT_DIR = "ttm_fewshot_quantile_plots"
os.makedirs(PLOT_DIR, exist_ok=True)

max_idx = len(dset_test_fs) - 1
plot_indices = [i for i in [0, 5, 10] if i <= max_idx]
if not plot_indices:
    plot_indices = [0]

plot_predictions(
    model=trainer.model,
    dset=dset_test_fs,
    plot_dir=PLOT_DIR,
    plot_prefix="amk_fewshot_quantile_rainfall",
    indices=plot_indices,
    channel=0,
)

print("\n✅ Few-shot quantile rainfall plots saved to:", PLOT_DIR)


Binary Code for APP Deployment

In [ ]:
# ======================================================
# BINARY RAIN / NO-RAIN DECISION
# Based on FEW-SHOT + QUANTILE predictions
# ======================================================

import numpy as np

# preds comes from:
# preds = trainer.predict(dset_test_fs).predictions[0]
# Shape: (samples, 7, 3)

# Channel 0 = Rainfall
rain_preds = preds[:, :, 0]  # (samples, 7)

RAIN_THRESHOLD = 0.2  # mm

binary_results = []

for i, window in enumerate(rain_preds):
    will_rain = np.any(window >= RAIN_THRESHOLD)

    result = {
        "window_id": int(i),
        "max_predicted_rain_mm": float(window.max()),
        "rain_decision": "Most likely to rain" if will_rain else "Unlikely to rain"
    }
    binary_results.append(result)

# Show first few decisions
for r in binary_results[:5]:
    print(r)


In [ ]:
!pip install fastapi uvicorn pyngrok


In [ ]:
%%writefile app.py
from fastapi import FastAPI

app = FastAPI()

@app.get("/rain_prediction")
def rain_prediction():
    # Replace this later with your REAL model output
    predicted_rain = [0.0, 0.1, 0.3, 1.2, 0.0, 0.0, 0.0]

    RAIN_THRESHOLD = 0.2
    will_rain = any(r >= RAIN_THRESHOLD for r in predicted_rain)

    return {
        "prediction": "Most likely to rain" if will_rain else "Unlikely to rain",
        "max_rain_mm": max(predicted_rain),
        "next_7_days": predicted_rain
    }


In [ ]:
!nohup uvicorn app:app --host 0.0.0.0 --port 8000 > uvicorn.log 2>&1 &



In [ ]:
!ps aux | grep uvicorn


In [ ]:
!wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64 -O cloudflared
!chmod +x cloudflared
!./cloudflared --version


In [ ]:
!nohup ./cloudflared tunnel --url http://localhost:8000 > cloudflared.log 2>&1 &


In [ ]:
!grep -o "https://[a-zA-Z0-9.-]*trycloudflare.com" -m 1 cloudflared.log


RESTART'


In [ ]:
!pkill -f uvicorn


In [ ]:
%%writefile app.py
from fastapi import FastAPI

app = FastAPI()

@app.get("/rain_prediction")
def rain_prediction():
    predicted_rain = [0.0, 0.1, 0.3, 1.2, 0.0, 0.0, 0.0]
    RAIN_THRESHOLD = 0.2
    will_rain = any(r >= RAIN_THRESHOLD for r in predicted_rain)

    return {
        "prediction": "Most likely to rain" if will_rain else "Unlikely to rain",
        "max_rain_mm": max(predicted_rain),
        "next_7_days": predicted_rain
    }


In [ ]:
!nohup uvicorn app:app --host 0.0.0.0 --port 8000 > uvicorn.log 2>&1 &


In [ ]:
!tail -n 60 uvicorn.log


In [ ]:
import requests
r = requests.get("http://127.0.0.1:8000/rain_prediction", timeout=5)
print("Local status:", r.status_code)
print("Local text:", r.text)


In [ ]:
!pkill -f cloudflared
!nohup ./cloudflared tunnel --url http://localhost:8000 > cloudflared.log 2>&1 &
!grep -o "https://[a-zA-Z0-9.-]*trycloudflare.com" -m 1 cloudflared.log


In [ ]:
!ps aux | grep uvicorn


In [ ]:
!tail -n 50 uvicorn.log


In [ ]:
!pkill -f cloudflared


In [ ]:
!nohup ./cloudflared tunnel --url http://localhost:8000 > cloudflared.log 2>&1 &


In [ ]:
!tail -n 40 cloudflared.log


In [ ]:
!grep -o "https://nicholas-folder-responded-imperial.trycloudflare.com" cloudflared.log | head -n 1


In [ ]:
import requests

url = "https://nicholas-folder-responded-imperial.trycloudflare.com/rain_prediction"
r = requests.get(url, timeout=10)

print("Status:", r.status_code)
print("Text:", r.text[:300])


In [ ]:
print(r.json())


In [ ]:
print(requests.get("http://127.0.0.1:8000/rain_prediction").text)



In [ ]:
%%writefile app.py
from fastapi import FastAPI

app = FastAPI()

@app.get("/")
def root():
    return {"status": "API is running"}

@app.get("/rain_prediction")
def rain_prediction():
    predicted_rain = [0.0, 0.1, 0.3, 1.2, 0.0, 0.0, 0.0]
    RAIN_THRESHOLD = 0.2
    will_rain = any(r >= RAIN_THRESHOLD for r in predicted_rain)

    return {
        "prediction": "Most likely to rain" if will_rain else "Unlikely to rain",
        "max_rain_mm": max(predicted_rain),
        "next_7_days": predicted_rain
    }


In [ ]:
!ls -l app.py


In [ ]:
!pkill -f uvicorn
!nohup uvicorn app:app --host 0.0.0.0 --port 8000 > uvicorn.log 2>&1 &


In [ ]:
!tail -n 50 uvicorn.log


In [ ]:
import requests

r = requests.get("http://127.0.0.1:8000/rain_prediction", timeout=5)
print("Local status:", r.status_code)
print("Local text:", r.text)


In [ ]:
!pkill -f cloudflared
!nohup ./cloudflared tunnel --url http://localhost:8000 > cloudflared.log 2>&1 &
!grep -o "https://[a-zA-Z0-9.-]*trycloudflare.com" cloudflared.log | head -n 1


In [ ]:
!tail -n 50 cloudflared.log


In [ ]:
import requests
url = "https://tions-meanwhile-neural-quantity.trycloudflare.com/rain_prediction"
r = requests.get(url, timeout=10)
print(r.status_code)
print(r.text)

In [ ]:
!pip install fastapi uvicorn


In [ ]:
!pkill -9 -f uvicorn


In [ ]:
!lsof -i :8000


In [ ]:
!nohup uvicorn app:app --host 0.0.0.0 --port 8000 > uvicorn.log 2>&1 &


In [ ]:
!tail -n 20 uvicorn.log


In [ ]:
import requests
r = requests.get("http://127.0.0.1:8000/rain_prediction")
print(r.status_code)
print(r.json())


In [ ]:
from fastapi import FastAPI

app = FastAPI()

@app.get("/")
def root():
    return {"status": "API is running"}

@app.get("/rain_prediction")
def rain_prediction():
    predicted_rain = [0.0, 0.1, 0.3, 1.2, 0.0, 0.0, 0.0]
    will_rain = any(r >= 0.2 for r in predicted_rain)

    return {
        "prediction": "Most likely to rain" if will_rain else "Unlikely to rain",
        "max_rain_mm": max(predicted_rain),
        "next_7_days": predicted_rain
    }


In [ ]:
!pip install fastapi uvicorn


In [ ]:
%%writefile requirements.txt
fastapi==0.110.0
uvicorn[standard]==0.29.0


In [ ]:
!ls -l app.py requirements.txt


In [ ]:
!zip api_backend.zip app.py requirements.txt


ALL REGION


In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error, confusion_matrix, precision_recall_fscore_support, accuracy_score

from sktime.forecasting.ttm import TinyTimeMixerForecaster
from sktime.forecasting.base import ForecastingHorizon

# === Load data ===
CSV_PATH = "content/2NewCombined.csv"   # upload your file to Colab, or change path
df = pd.read_csv(CSV_PATH)

# Your Date column is day-first sometimes (e.g. 13/4/2025), so use mixed parsing:
df["Date"] = pd.to_datetime(df["Date"], format="mixed", dayfirst=True)
df = df.sort_values("Date").reset_index(drop=True)
df = df.set_index("Date")

print("Date range:", df.index.min(), "→", df.index.max())
print("Rows, Cols:", df.shape)


In [ ]:
REGIONS = {
    "Admiralty": "Admiralty",
    "Ang Mo Kio": "Ang Mo Kio",
    "Changi": "Changi",
    "Sentosa Island": "Sentosa Island",
    "Tuas South": "Tuas South",
}

def col(region_suffix: str, prefix: str) -> str:
    """Build exact column name safely."""
    return f"{prefix}_{region_suffix}"

def prepare_region_xy(df: pd.DataFrame, region_suffix: str):
    """
    y = Daily Rainfall Total (mm)_<region>
    X = other features for same region (wind/temp + peak rain intensities)
    """
    y_col = col(region_suffix, "Daily Rainfall Total (mm)")
    if y_col not in df.columns:
        raise ValueError(f"Missing target column: {y_col}")

    # Use ALL other columns that end with _<region> as exogenous
    region_cols = [c for c in df.columns if c.endswith(f"_{region_suffix}")]
    x_cols = [c for c in region_cols if c != y_col]

    y = df[y_col].astype(float)
    X = df[x_cols].astype(float)

    # Fill missing values (simple + robust)
    y = y.fillna(method="ffill").fillna(method="bfill")
    X = X.fillna(method="ffill").fillna(method="bfill")

    return y, X, y_col, x_cols

def make_binary_rain(y_mm: pd.Series, threshold_mm: float = 1.0) -> pd.Series:
    """Convert rainfall (mm) into rain/no-rain label."""
    return (y_mm >= threshold_mm).astype(int)


In [ ]:
# === Evaluation period ===
TEST_START = "2025-07-01"
TEST_END   = "2025-09-30"

# Train uses everything before TEST_START
# Test uses [TEST_START .. TEST_END]

RAIN_THRESHOLD_MM = 1.0   # choose 0.0 if you want "any rain", but 1.0 is less noisy

alphas = [0.1, 0.5, 0.9]  # quantiles


In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error, confusion_matrix, precision_recall_fscore_support, accuracy_score
from sktime.forecasting.ttm import TinyTimeMixerForecaster
from sktime.forecasting.base import ForecastingHorizon

REGIONS = {
    "Admiralty": "Admiralty",
    "Ang Mo Kio": "Ang Mo Kio",
    "Changi": "Changi",
    "Sentosa Island": "Sentosa Island",
    "Tuas South": "Tuas South",
}

def col(region_suffix: str, prefix: str) -> str:
    return f"{prefix}_{region_suffix}"

def prepare_region_xy(df: pd.DataFrame, region_suffix: str):
    y_col = col(region_suffix, "Daily Rainfall Total (mm)")
    region_cols = [c for c in df.columns if c.endswith(f"_{region_suffix}")]
    x_cols = [c for c in region_cols if c != y_col]

    y = df[y_col].astype(float).ffill().bfill()
    X = df[x_cols].astype(float).ffill().bfill()
    return y, X

def make_binary_rain(y_mm: pd.Series, threshold_mm: float = 1.0) -> pd.Series:
    return (y_mm >= threshold_mm).astype(int)

TEST_START = "2025-07-01"
TEST_END   = "2025-09-30"
RAIN_THRESHOLD_MM = 1.0

all_metrics = []
all_outputs = []

for region_name, region_suffix in REGIONS.items():
    print("\n==============================")
    print("Region:", region_name)

    y, X = prepare_region_xy(df, region_suffix)

    train_end = pd.to_datetime(TEST_START) - pd.Timedelta(days=1)
    y_train = y.loc[:train_end]
    X_train = X.loc[y_train.index]

    y_test = y.loc[TEST_START:TEST_END]
    X_test = X.loc[y_test.index]

    if len(y_train) < 60 or len(y_test) < 10:
        print("⚠️ Not enough data, skipping.")
        continue

    # Horizon for test
    fh = ForecastingHorizon(np.arange(1, len(y_test) + 1), is_relative=True)

    forecaster = TinyTimeMixerForecaster(model_path="ibm/TTM", revision="main", fit_strategy="minimal")
    forecaster.fit(y_train, X=X_train, fh=fh)

    # ✅ Point forecast (q50)
    y_pred = forecaster.predict(X=X_test)  # returns Series aligned to fh steps

    # Align prediction to y_test index
    y_pred = pd.Series(y_pred.to_numpy(), index=y_test.index, name="q50_mm")

    # --- Empirical residual quantiles from TRAIN (for uncertainty band) ---
    # Get in-sample predictions by rolling 1-step ahead within train (cheap approximation)
    # We'll do a simple approach: use residual distribution from last N days of train using naive baseline
    # Better: compute residuals on a small validation split inside train.
    # We'll do a clean validation split:
    split = int(len(y_train) * 0.85)
    y_tr = y_train.iloc[:split]
    X_tr = X_train.iloc[:split]
    y_va = y_train.iloc[split:]
    X_va = X_train.iloc[split:]

    fh_va = ForecastingHorizon(np.arange(1, len(y_va) + 1), is_relative=True)
    f_va = TinyTimeMixerForecaster(model_path="ibm/TTM", revision="main", fit_strategy="minimal")
    f_va.fit(y_tr, X=X_tr, fh=fh_va)
    va_pred = f_va.predict(X=X_va)
    va_pred = pd.Series(va_pred.to_numpy(), index=y_va.index)

    residuals = (y_va - va_pred).dropna()
    r_q10 = residuals.quantile(0.10)
    r_q90 = residuals.quantile(0.90)

    q10 = (y_pred + r_q10).clip(lower=0.0)
    q90 = (y_pred + r_q90).clip(lower=0.0)

    # Metrics (continuous on q50)
    mae = mean_absolute_error(y_test.values, y_pred.values)
    rmse = np.sqrt(mean_squared_error(y_test.values, y_pred.values))


    # Binary metrics (for app)
    y_true_bin = make_binary_rain(y_test, RAIN_THRESHOLD_MM)
    y_pred_bin = make_binary_rain(y_pred, RAIN_THRESHOLD_MM)

    acc = accuracy_score(y_true_bin, y_pred_bin)
    prec, rec, f1, _ = precision_recall_fscore_support(y_true_bin, y_pred_bin, average="binary", zero_division=0)
    cm = confusion_matrix(y_true_bin, y_pred_bin)

    all_metrics.append({
        "region": region_name,
        "MAE_mm(q50)": mae,
        "RMSE_mm(q50)": rmse,
        "Accuracy(bin)": acc,
        "Precision(bin)": prec,
        "Recall(bin)": rec,
        "F1(bin)": f1,
        "TN_FP_FN_TP": cm.ravel().tolist() if cm.size == 4 else cm.tolist(),
        "residual_q10": float(r_q10),
        "residual_q90": float(r_q90),
    })

    out = pd.DataFrame({
        "Date": y_test.index,
        "actual_mm": y_test.values,
        "q10_mm": q10.values,
        "q50_mm": y_pred.values,
        "q90_mm": q90.values,
        "actual_rain_bin": y_true_bin.values,
        "pred_rain_bin": y_pred_bin.values,
        "region": region_name
    })

    out_path = f"/content/backtest_{region_name.replace(' ', '_')}_JulSep2025.csv"
    out.to_csv(out_path, index=False)
    print("Saved:", out_path)

    all_outputs.append(out)

metrics_df = pd.DataFrame(all_metrics)
metrics_df.to_csv("/content/ttm_metrics_summary.csv", index=False)
print("\nSaved: /content/ttm_metrics_summary.csv")

combined = pd.concat(all_outputs, ignore_index=True)
combined.to_csv("/content/backtest_all_regions_JulSep2025.csv", index=False)
print("Saved: /content/backtest_all_regions_JulSep2025.csv")

metrics_df


In [ ]:
import pandas as pd
import numpy as np
import json
from sktime.forecasting.ttm import TinyTimeMixerForecaster
from sktime.forecasting.base import ForecastingHorizon

# --- SETTINGS ---
RAIN_THRESHOLD_MM = 1.0  # >= 1.0mm considered "rain"
FORECAST_DAYS = 7

REGIONS = {
    "Admiralty": "Admiralty",
    "Ang Mo Kio": "Ang Mo Kio",
    "Changi": "Changi",
    "Sentosa Island": "Sentosa Island",
    "Tuas South": "Tuas South",
}

def col(region_suffix: str, prefix: str) -> str:
    return f"{prefix}_{region_suffix}"

def prepare_region_xy(df: pd.DataFrame, region_suffix: str):
    y_col = col(region_suffix, "Daily Rainfall Total (mm)")
    region_cols = [c for c in df.columns if c.endswith(f"_{region_suffix}")]
    x_cols = [c for c in region_cols if c != y_col]

    y = df[y_col].astype(float).ffill().bfill()
    X = df[x_cols].astype(float).ffill().bfill()
    return y, X

def forecast_7days_for_region(df: pd.DataFrame, metrics_df: pd.DataFrame, region_name: str, region_suffix: str):
    # Use "today" for display, but base_date must exist in df index to build history
    today = pd.Timestamp.today(tz=None).normalize()

    # base_date = last date available in dataset (or today if today exists)
    base_date = today if today in df.index else df.index.max()

    y, X = prepare_region_xy(df, region_suffix)

    # Train on all data up to base_date
    y_train = y.loc[:base_date]
    X_train = X.loc[y_train.index]

    # Future dates (7 days)
    future_idx = pd.date_range(base_date + pd.Timedelta(days=1), periods=FORECAST_DAYS, freq="D")

    # DEMO exogenous: repeat last known X
    last_x = X.loc[[base_date]].copy()
    X_future = pd.concat([last_x] * FORECAST_DAYS, ignore_index=True)
    X_future.index = future_idx

    fh = ForecastingHorizon(np.arange(1, FORECAST_DAYS + 1), is_relative=True)

    model = TinyTimeMixerForecaster(model_path="ibm/TTM", revision="main", fit_strategy="minimal")
    model.fit(y_train, X=X_train, fh=fh)

    # Point forecast (q50)
    pred = model.predict(X=X_future)
    q50 = pd.Series(pred.to_numpy(), index=future_idx, name="q50_mm").clip(lower=0.0)

    # Use your saved residual quantiles from metrics_df for q10/q90 bands
    row = metrics_df.loc[metrics_df["region"] == region_name].iloc[0]
    r_q10 = float(row["residual_q10"])
    r_q90 = float(row["residual_q90"])

    q10 = (q50 + r_q10).clip(lower=0.0)
    q90 = (q50 + r_q90).clip(lower=0.0)

    predictionInt = (q50 >= RAIN_THRESHOLD_MM).astype(int)

    out = pd.DataFrame({
        "region": region_name,
        "date": future_idx.strftime("%Y-%m-%d"),
        "q10_mm": q10.round(3).values,
        "q50_mm": q50.round(3).values,
        "q90_mm": q90.round(3).values,
        "predictionInt": predictionInt.values,  # 0/1 for FlutterFlow
        "base_date_used": str(base_date.date()),
        "today_display": str(today.date()),
    })
    return out

# ---- LOAD your metrics df ----
# You saved it as /content/ttm_metrics_summary.csv in your screenshot output
metrics_df = pd.read_csv("/content/ttm_metrics_summary.csv")

# ---- BUILD forecasts for all regions ----
all_out = []
for region_name, region_suffix in REGIONS.items():
    print("Forecasting:", region_name)
    out = forecast_7days_for_region(df, metrics_df, region_name, region_suffix)
    all_out.append(out)

forecast_df = pd.concat(all_out, ignore_index=True)

# Save CSV for checking
csv_path = "/content/flutterflow_7day_forecast.csv"
forecast_df.to_csv(csv_path, index=False)
print("Saved:", csv_path)

# Save JSON for FlutterFlow (grouped by region)
grouped = {}
for region_name in REGIONS.keys():
    region_rows = forecast_df[forecast_df["region"] == region_name].copy()
    grouped[region_name] = region_rows.drop(columns=["region"]).to_dict(orient="records")

json_path = "/content/flutterflow_7day_forecast.json"
with open(json_path, "w") as f:
    json.dump({
        "generatedAt": pd.Timestamp.now().isoformat(),
        "rainThresholdMm": RAIN_THRESHOLD_MM,
        "days": FORECAST_DAYS,
        "forecasts": grouped
    }, f, indent=2)

print("Saved:", json_path)

forecast_df.head(15)


In [ ]:
import pandas as pd
import numpy as np
from sktime.forecasting.ttm import TinyTimeMixerForecaster
from sktime.forecasting.base import ForecastingHorizon

# --- SETTINGS ---
RAIN_THRESHOLD_MM = 1.0  # rain if >= 1.0mm
REGIONS = {
    "Admiralty": "Admiralty",
    "Ang Mo Kio": "Ang Mo Kio",
    "Changi": "Changi",
    "Sentosa Island": "Sentosa Island",
    "Tuas South": "Tuas South",
}

def col(region_suffix: str, prefix: str) -> str:
    return f"{prefix}_{region_suffix}"

def prepare_region_xy(df: pd.DataFrame, region_suffix: str):
    y_col = col(region_suffix, "Daily Rainfall Total (mm)")
    region_cols = [c for c in df.columns if c.endswith(f"_{region_suffix}")]
    x_cols = [c for c in region_cols if c != y_col]

    y = df[y_col].astype(float).ffill().bfill()
    X = df[x_cols].astype(float).ffill().bfill()
    return y, X

# Load residual quantiles (from your earlier metrics output)
metrics_df = pd.read_csv("/content/ttm_metrics_summary.csv")
resid_map = metrics_df.set_index("region")[["residual_q10","residual_q90"]].to_dict("index")

base_date = df.index.max()  # last date in dataset (stable for demo)

rows = []

for region_name, region_suffix in REGIONS.items():
    print("Building 3-day for:", region_name)

    y, X = prepare_region_xy(df, region_suffix)

    # We can show yesterday actual because it exists in dataset:
    yesterday_date = base_date - pd.Timedelta(days=1)
    today_date = base_date
    tomorrow_date = base_date + pd.Timedelta(days=1)

    # Train on all data up to "today_date"
    y_train = y.loc[:today_date]
    X_train = X.loc[y_train.index]

    # Forecast only 1 step ahead (tomorrow) using repeated last X
    fh = ForecastingHorizon([1], is_relative=True)
    model = TinyTimeMixerForecaster(model_path="ibm/TTM", revision="main", fit_strategy="minimal")
    model.fit(y_train, X=X_train, fh=fh)

    # Exogenous for tomorrow: repeat last known X
    last_x = X.loc[[today_date]].copy()
    X_future = last_x.copy()
    X_future.index = [tomorrow_date]

    pred_tomorrow = float(model.predict(X=X_future).to_numpy()[0])
    pred_tomorrow = max(pred_tomorrow, 0.0)

    # For Yesterday/Today predicted values:
    # Use "q50_mm = actual_mm" as a simple baseline display OR
    # if you prefer model-based, you'd need a backtest prediction at those dates.
    # For a clean demo: show actual for yesterday/today, model forecast for tomorrow.

    actual_yesterday = float(y.loc[yesterday_date]) if yesterday_date in y.index else np.nan
    actual_today = float(y.loc[today_date]) if today_date in y.index else np.nan

    # Use residual bands around q50
    r10 = float(resid_map[region_name]["residual_q10"])
    r90 = float(resid_map[region_name]["residual_q90"])

    def make_row(dayIndex, date, q50, actual=None):
        q10 = max(q50 + r10, 0.0)
        q90 = max(q50 + r90, 0.0)
        predInt = 1 if q50 >= RAIN_THRESHOLD_MM else 0
        return {
            "region": region_name,
            "dayIndex": dayIndex,          # 0=yesterday, 1=today, 2=tomorrow
            "label": ["Yesterday","Today","Tomorrow"][dayIndex],
            "date": pd.to_datetime(date).strftime("%Y-%m-%d"),
            "actual_mm": None if actual is None or np.isnan(actual) else round(float(actual), 3),
            "q10_mm": round(float(q10), 3),
            "q50_mm": round(float(q50), 3),
            "q90_mm": round(float(q90), 3),
            "predictionInt": int(predInt),
        }

    # Yesterday: use actual as q50 (so it reflects rainfall that happened)
    rows.append(make_row(0, yesterday_date, actual_yesterday, actual=actual_yesterday))

    # Today: use actual as q50 (reflects today's observed value in dataset)
    rows.append(make_row(1, today_date, actual_today, actual=actual_today))

    # Tomorrow: use model forecast
    rows.append(make_row(2, tomorrow_date, pred_tomorrow, actual=None))

out_df = pd.DataFrame(rows)
out_path = "/content/flutterflow_3day_forecast.csv"
out_df.to_csv(out_path, index=False)
print("Saved:", out_path)

out_df.head(15)


FLUTTERFLOW FINAL

In [12]:
import pandas as pd
import numpy as np
from sktime.forecasting.ttm import TinyTimeMixerForecaster
from sktime.forecasting.base import ForecastingHorizon

df = pd.read_csv("/content/2NewCombined.csv")
df["Date"] = pd.to_datetime(df["Date"], format="mixed", dayfirst=True)
df = df.sort_values("Date").reset_index(drop=True)
df = df.set_index("Date")
print(df.index.min(), "→", df.index.max())



ModuleNotFoundError: No module named 'sktime'

In [ ]:
REGIONS = {
    "Admiralty": "Admiralty",
    "Ang Mo Kio": "Ang Mo Kio",
    "Changi": "Changi",
    "Sentosa Island": "Sentosa Island",
    "Tuas South": "Tuas South",
}

RAIN_THRESHOLD_MM = 1.0

def col(region_suffix: str, prefix: str) -> str:
    return f"{prefix}_{region_suffix}"

def prepare_region_xy(df: pd.DataFrame, region_suffix: str):
    y_col = col(region_suffix, "Daily Rainfall Total (mm)")
    region_cols = [c for c in df.columns if c.endswith(f"_{region_suffix}")]
    x_cols = [c for c in region_cols if c != y_col]

    y = df[y_col].astype(float).ffill().bfill()
    X = df[x_cols].astype(float).ffill().bfill()
    return y, X


In [ ]:
# If you want to force the demo day (e.g., 2026-02-06), set it here:
TODAY_OVERRIDE = None  # e.g. "2026-02-06"

today = pd.to_datetime(TODAY_OVERRIDE) if TODAY_OVERRIDE else pd.Timestamp.today().normalize()
yesterday = today - pd.Timedelta(days=1)
tomorrow = today + pd.Timedelta(days=1)

base_date = df.index.max()  # last known date in dataset (model base)

rows = []

for region_name, region_suffix in REGIONS.items():
    print("Forecasting:", region_name)
    y, X = prepare_region_xy(df, region_suffix)

    # Train up to base_date
    y_train = y.loc[:base_date]
    X_train = X.loc[y_train.index]

    # Forecast next 3 steps from base_date
    fh = ForecastingHorizon([1, 2, 3], is_relative=True)
    model = TinyTimeMixerForecaster(model_path="ibm/TTM", revision="main", fit_strategy="minimal")
    model.fit(y_train, X=X_train, fh=fh)

    # Repeat last known X for the next 3 days
    last_x = X.loc[[base_date]].copy()
    fut_idx = pd.date_range(base_date + pd.Timedelta(days=1), periods=3, freq="D")
    X_future = pd.concat([last_x] * 3, ignore_index=True)
    X_future.index = fut_idx

    preds = model.predict(X=X_future).to_numpy().astype(float)
    preds = np.clip(preds, 0.0, None)

    def make_row(dayIndex, label, display_date, q50):
        predInt = 1 if q50 >= RAIN_THRESHOLD_MM else 0
        return {
            "region": region_name,
            "dayIndex": dayIndex,  # -1,0,1
            "label": label,
            "date": pd.to_datetime(display_date).strftime("%Y-%m-%d"),
            "q50_mm": round(float(q50), 3),
            "predictionInt": int(predInt),
            "base_date_used": str(base_date.date())
        }

    # Map to your UI's indexing
    rows.append(make_row(-1, "Yesterday", yesterday, preds[0]))
    rows.append(make_row( 0, "Today",     today,     preds[1]))
    rows.append(make_row( 1, "Tomorrow",  tomorrow,  preds[2]))

out_df = pd.DataFrame(rows)
out_path = "/content/flutterflow_yesterday_today_tomorrow.json"
out_df.to_csv(out_path, index=False)
print("\nSaved:", out_path)
out_df.head(15)


In [ ]:
from fastapi import FastAPI, HTTPException
from fastapi.middleware.cors import CORSMiddleware
import pandas as pd

CSV_PATH = "flutterflow_yesterday_today_tomorrow.csv"

app = FastAPI()

# Allow FlutterFlow/web requests
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

# Load once at startup
df = pd.read_csv(CSV_PATH)

def pick_region(region: str):
    sub = df[df["region"].str.strip().str.lower() == region.strip().lower()].copy()
    if sub.empty:
        raise HTTPException(status_code=404, detail=f"Region not found: {region}")

    # Ensure dayIndex is int
    sub["dayIndex"] = sub["dayIndex"].astype(int)

    def row_for(idx):
        r = sub[sub["dayIndex"] == idx]
        if r.empty:
            raise HTTPException(status_code=500, detail=f"Missing dayIndex {idx} for region {region}")
        r = r.iloc[0]
        return {
            "date": str(r["date"]),
            "q50_mm": float(r["q50_mm"]),
            "predictionInt": int(r["predictionInt"]),
        }

    return {
        "region": region,
        "yesterday": row_for(-1),
        "today": row_for(0),
        "tomorrow": row_for(1),
    }

@app.get("/forecast3")
def forecast3(region: str):
    return pick_region(region)


7 Day prediction Admiralty

In [22]:
!pip install sktime



In [24]:
import pandas as pd
import numpy as np

from sktime.forecasting.ttm import TinyTimeMixerForecaster
from sktime.forecasting.base import ForecastingHorizon


In [25]:
df = pd.read_csv("/content/2NewCombined.csv")

df["Date"] = pd.to_datetime(df["Date"], format="mixed", dayfirst=True)
df = df.sort_values("Date").reset_index(drop=True)
df = df.set_index("Date")

print("Date range:", df.index.min(), "→", df.index.max())


Date range: 2025-04-01 00:00:00 → 2025-09-30 00:00:00


In [26]:
y = (
    df["Daily Rainfall Total (mm)_Admiralty"]
    .fillna(0)
    .astype(float)
)



In [27]:
X = df[
    [
        "Mean Temperature (°C)_Ang Mo Kio",
        "Mean Wind Speed (km/h)_Ang Mo Kio"
    ]
].fillna(method="ffill")



/tmp/ipython-input-253601793.py:6: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  ].fillna(method="ffill")


In [28]:
fh = ForecastingHorizon(
    np.arange(1, 8),   # next 7 days
    is_relative=True
)



In [29]:
forecaster = TinyTimeMixerForecaster()


In [ ]:
import json

response = {
    "region": "Admiralty",
    "generated_at": today.isoformat(),
    "forecast": results
}

print(json.dumps(response, indent=2))


In [30]:
forecaster.fit(
    y=y,
    X=X,
    fh=fh
)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/3.24M [00:00<?, ?B/s]

TinyTimeMixerForecaster()

In [31]:
y_pred = forecaster.predict(
    fh=fh,
    X=X
)

print(y_pred)


Date
2025-10-01    11.580156
2025-10-02     9.810423
2025-10-03     8.048119
2025-10-04     7.453722
2025-10-05     8.199714
2025-10-06     7.690293
2025-10-07     7.192711
Freq: D, Name: Daily Rainfall Total (mm)_Admiralty, dtype: float32


In [32]:
today = pd.Timestamp.today().normalize()

RAIN_THRESHOLD_MM = 1.0
results = []

for i, rain_mm in enumerate(y_pred):
    rain_mm = max(float(rain_mm), 0.0)
    will_rain = 1 if rain_mm >= RAIN_THRESHOLD_MM else 0

    results.append({
        "dayIndex": i,  # 0 = today
        "date": (today + pd.Timedelta(days=i)).strftime("%Y-%m-%d"),
        "rain_mm": round(rain_mm, 2),
        "prediction": will_rain,
        "predictionText": (
            "Most likely to rain" if will_rain
            else "Not likely to rain"
        )
    })

forecast_df = pd.DataFrame(results)
forecast_df


,dayIndex,date,rain_mm,prediction,predictionText
0,0,2026-02-01,11.58,1,Most likely to rain
1,1,2026-02-02,9.81,1,Most likely to rain
2,2,2026-02-03,8.05,1,Most likely to rain
3,3,2026-02-04,7.45,1,Most likely to rain
4,4,2026-02-05,8.20,1,Most likely to rain
5,5,2026-02-06,7.69,1,Most likely to rain
6,6,2026-02-07,7.19,1,Most likely to rain


In [33]:
import json

response = {
    "region": "Admiralty",
    "generated_at": today.strftime("%Y-%m-%d"),
    "forecast": results
}

print(json.dumps(response, indent=2))


{
  "region": "Admiralty",
  "generated_at": "2026-02-01",
  "forecast": [
    {
      "dayIndex": 0,
      "date": "2026-02-01",
      "rain_mm": 11.58,
      "prediction": 1,
      "predictionText": "Most likely to rain"
    },
    {
      "dayIndex": 1,
      "date": "2026-02-02",
      "rain_mm": 9.81,
      "prediction": 1,
      "predictionText": "Most likely to rain"
    },
    {
      "dayIndex": 2,
      "date": "2026-02-03",
      "rain_mm": 8.05,
      "prediction": 1,
      "predictionText": "Most likely to rain"
    },
    {
      "dayIndex": 3,
      "date": "2026-02-04",
      "rain_mm": 7.45,
      "prediction": 1,
      "predictionText": "Most likely to rain"
    },
    {
      "dayIndex": 4,
      "date": "2026-02-05",
      "rain_mm": 8.2,
      "prediction": 1,
      "predictionText": "Most likely to rain"
    },
    {
      "dayIndex": 5,
      "date": "2026-02-06",
      "rain_mm": 7.69,
      "prediction": 1,
      "predictionText": "Most likely to rain"
    },
 

FileNotFoundError: Cannot find file: admiralty_7day_rain_forecast.csv

Admiralty Code

In [35]:
import pandas as pd
import numpy as np
from sktime.forecasting.ttm import TinyTimeMixerForecaster
from sktime.forecasting.base import ForecastingHorizon

# Load data
df = pd.read_csv("/content/2NewCombined.csv")
df["Date"] = pd.to_datetime(df["Date"], format="mixed", dayfirst=True)
df = df.sort_values("Date").reset_index(drop=True)
df = df.set_index("Date")

# TARGET: Admiralty rainfall ONLY
y = df["Daily Rainfall Total (mm)_Admiralty"].fillna(0).astype(float)

# Forecast horizon: next 7 days
fh = ForecastingHorizon(np.arange(1, 8), is_relative=True)

# Model
forecaster = TinyTimeMixerForecaster()
forecaster.fit(y=y, fh=fh)

# Predict
y_pred = forecaster.predict(fh=fh)

# Format output
today = pd.Timestamp.today().normalize()
results = []

for i, rain_mm in enumerate(y_pred):
    rain_mm = max(float(rain_mm), 0.0)
    prediction = 1 if rain_mm >= 1.0 else 0

    results.append({
        "dayIndex": i,
        "date": (today + pd.Timedelta(days=i)).strftime("%Y-%m-%d"),
        "rain_mm": round(rain_mm, 2),
        "prediction": prediction,
        "predictionText": (
            "Most likely to rain" if prediction
            else "Not likely to rain"
        )
    })

forecast_df = pd.DataFrame(results)
forecast_df


/usr/local/lib/python3.12/dist-packages/huggingface_hub/file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


,dayIndex,date,rain_mm,prediction,predictionText
0,0,2026-02-01,11.58,1,Most likely to rain
1,1,2026-02-02,9.81,1,Most likely to rain
2,2,2026-02-03,8.05,1,Most likely to rain
3,3,2026-02-04,7.45,1,Most likely to rain
4,4,2026-02-05,8.20,1,Most likely to rain
5,5,2026-02-06,7.69,1,Most likely to rain
6,6,2026-02-07,7.19,1,Most likely to rain


In [37]:
forecast_df.to_csv("admiralty_7day_rain_forecast.json", index=False)
